In [1]:
import sys
sys.path.append("../../")
import PolyMID
import pandas as pd
from PolyMID.AnalyzeSpectra import fragment_library
import numpy as np

from pdb import set_trace

In [2]:
# Get the metabolite library
file_directory = './'
metabolite_dict,metabolite_list = fragment_library.fragment_library(file_directory=file_directory,Full_NC=False)

    pyruvate_mox_tbdms
    lactate_2tbdms
    alanine_2tbdms
    glycine_2tbdms
    3-hydroxypropionicacid_2tbdms
    valine_2tbdms
    norvaline_2tbdms
    beta-alanine_2tbdms
    leucine_2tbdms
    isoleucine_2tbdms
    succinate_2tbdms
    uracil_2tbdms
    fumarate_2tbdms
    serine_3tbdms
    threonine_3tbdms
    taurine_2tbdms
    methionine_2tbdms
    alpha-ketoglutarate_mox_2tbdms
    malate_3tbdms
    phenylalanine_2tbdms
    aspartate_3tbdms
    proline_MOX_2tbdms
    glutamate_3tbdms
    asparagine_3tbdms
    lysine_3tbdms
    glutamine_3tbdms
    arginine_4tbdms
    citrate_4tbdms
    tyrosine_3tbdms
    histidine_3tbdms
    tryptophan_3tbdms
    glucose
    glucose_clean


../../PolyMID/Classes/AtomClass.py:45: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (self.symbol not in self.HighRes) & ('all' not in self.HighRes):
../../PolyMID/Classes/AtomClass.py:55: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if ((self.symbol in self.HighRes)|('all' in self.HighRes)) & (self.symbol != self.Tracer.LabeledElement):


In [24]:
# Fill in the MIDs for the unlabeled compounds of interest
Standard_MID_DF = pd.read_csv('Internal_Standards.txt', delimiter='\t', index_col=0)
fragments = Standard_MID_DF.columns
MID_length = 15

for met_key in metabolite_dict.keys():
    for frag_key in metabolite_dict[met_key]['fragments'].keys():
        if frag_key in fragments:
            Natural_MID = metabolite_dict[met_key]['fragments'][frag_key]['natural_mid'][0:MID_length]
            frag_key_position = Standard_MID_DF.columns.get_loc(frag_key)
            Standard_MID_DF.iloc[0:MID_length,frag_key_position] = Natural_MID
            Standard_MID_DF.iloc[MID_length:2*MID_length,frag_key_position] = np.zeros(MID_length)

In [30]:
# Fill in the MIDs for the labeled compounds of interest (internal standards)

# Open the file with the uncorrected MIDs from the Integrate() function
#   Note the last column of this file needs to be named 'predicted' manually
Labeled_Standard_DF = pd.read_csv('Internal_Standards_Uncorrected.txt', delimiter='\t', index_col=0)

MID_indices = [False]*len(Labeled_Standard_DF.index)
for i in range(len(Labeled_Standard_DF.index)):
    MID_Condition = (str(Labeled_Standard_DF.index[i])[-2] == 'M') | (str(Labeled_Standard_DF.index[i])[-3] == 'M')
    MID_indices[i] = MID_Condition

MID_DF = Labeled_Standard_DF[MID_indices].copy()
MID_DF.index = [MID_DF.index[i][0:(MID_DF.index[i].find('_')+4)] for i in range(len(MID_DF.index))]
MID_DF.drop('predicted',axis=1,inplace=True)
MID_DF['average'] = MID_DF.iloc[:, 0:3].mean(axis=1)

for frag in fragments:
    frag_indices = MID_DF.index ==frag
    frag_MID = MID_DF.loc[frag_indices,'average']
    frag_MID = np.array(frag_MID)
    if len(frag_MID) > MID_length:
        frag_MID = frag_MID[0:MID_length]
    frag_position = Standard_MID_DF.columns.get_loc(frag)
    Standard_MID_DF.iloc[MID_length:(MID_length+len(frag_MID)),frag_position] = frag_MID

# Print the filled in data frame
Standard_MID_DF.to_csv('Internal_Standards_Filled.txt', sep='\t', index=True, header=True)

In [ ]:
# need to manually add the MID for labeled and unlabeled glucose fragment
# need to deconvolute the glucose peak
# need to have a script that uses HPA as an internal standard for lactate

# supress the library saving as a pickle file
# have the library entries extended for intengration only by the number of nitrogens in the metabolite when Full_NC is True